In [1]:
import pandas as pd
from tqdm import tqdm
from tmdbv3api import Collection

import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import select_statement_to_df, get_from_table

In [2]:
all_collections_statement = """

SELECT COLLECTION_ID, COLLECTION_NAME, COUNT(*) AS FILM_COUNT
FROM FILM_COLLECTIONS
WHERE COLLECTION_ID != -1
GROUP BY COLLECTION_ID

"""

In [3]:
all_collections_df = select_statement_to_df(all_collections_statement)
all_collections_df

,COLLECTION_ID,COLLECTION_NAME,FILM_COUNT
0,10,Star Wars Collection,9
1,84,Indiana Jones Collection,5
2,119,The Lord of the Rings Collection,3
3,131,Three Colors Collection,3
4,151,Star Trek: The Original Series Collection,6
...,...,...,...
861,1105385,Tom Cody Collection,1
862,1117396,Death Trilogy,2
863,1118142,Grindhouse Collection,2
864,1118596,The Pope's Exorcist,1


In [4]:
def check_if_tmdb_id_ingested(tmdb_id):
    film_ingested = 0
    if len(select_statement_to_df('SELECT FILM_ID FROM TMDB_ID WHERE TMDB_ID = {}'.format(tmdb_id)).values) == 1:
        film_ingested = 1
    return film_ingested

In [14]:
def get_missing_films(collection_id):
    collection = Collection()
    collection_details = collection.details(collection_id)
    tmp_df = pd.DataFrame([{'TMDB_ID': x.id, 'FILM_TITLE': x.title, 'FILM_YEAR': x['release_date'][:4], 'COLLECTION_ID': collection_id} for x in collection_details['parts']])
    tmp_df['MISSING'] = 1 - tmp_df['TMDB_ID'].apply(check_if_tmdb_id_ingested)
    return tmp_df

In [15]:
all_collection_films = []
bad_collection_ids = []
for collection_id in tqdm(all_collections_df['COLLECTION_ID']):
    try:
        all_collection_films.append(get_missing_films(collection_id))
    except:
        bad_collection_ids.append(collection_id)

100%|██████████| 866/866 [01:18<00:00, 10.97it/s]


In [16]:
bad_collection_ids

[846090, 1117396, 1118142, 1118596]

In [17]:
all_collection_films_df = pd.concat(all_collection_films).reset_index(drop=True)
all_collection_films_df

,TMDB_ID,FILM_TITLE,FILM_YEAR,COLLECTION_ID,MISSING
0,11,Star Wars,1977,10,0
1,1891,The Empire Strikes Back,1980,10,0
2,1892,Return of the Jedi,1983,10,0
3,1893,Star Wars: Episode I - The Phantom Menace,1999,10,0
4,1894,Star Wars: Episode II - Attack of the Clones,2002,10,0
...,...,...,...,...,...
2765,1811,Nowhere,1997,1103488,0
2766,14746,Streets of Fire,1984,1105385,0
2767,105040,Road to Hell,2008,1105385,1
2768,619264,The Platform,2019,1126392,0


In [18]:
MISSING = all_collection_films_df[all_collection_films_df['MISSING'] == 1].reset_index(drop=True)

In [19]:
len(MISSING)

734

In [20]:
round = -1
increment = 50

In [21]:
round += 1
MISSING.head(increment*(round+1)).tail(increment)

,TMDB_ID,FILM_TITLE,FILM_YEAR,COLLECTION_ID,MISSING
0,755679,Fast X: Part 2,,9485,1
1,487560,Last Friday,,43563,1
2,983058,National Treasure 3,,52984,1
3,47743,The Saint in New York,1938,86224,1
4,45571,The Saint's Double Trouble,1940,86224,1
5,15440,Cold Prey II,2008,86553,1
6,56937,Cold Prey III,2010,86553,1
7,31723,Women in Trouble,2009,86668,1
8,726739,Cats & Dogs 3: Paws Unite,2020,87014,1
9,39691,Cats & Dogs: The Revenge of Kitty Galore,2010,87014,1
